In [1]:
import geopandas as gpd
import shapely
from getpass import getpass

import sys
import copy
sys.path.append("../Code")

from helper_functions import *
from loading_data import *
from algorithms import *

WelkomCorne!


In [ ]:
dfob= get_dataframe("""
                    SELECT bk_votpand_cluster, COUNT(*)
                    FROM proj_afval_netwerk.rel_votpand_cluster_verblijfsobject
                    GROUP BY bk_votpand_cluster
                    """)

In [ ]:
dfob['split'] = dfob['bk_votpand_cluster'].str.split('~')
dfob['bag'] = dfob['split'].apply(lambda x: x[0]).astype('int64')
dfob['x'] = dfob['split'].apply(lambda x: x[1]).astype('float').round().astype('int')
dfob['y'] = dfob['split'].apply(lambda x: x[2]).astype('float').round().astype('int')
dfob = dfob.drop(['split'], axis=1)

In [ ]:
df_afstandn2 = get_dataframe("""
                                SELECT *
                                FROM proj_afval_netwerk.afv_rel_nodes_poi
                                """)

In [ ]:
df_afstandn2['split'] = df_afstandn2['bk_afv_rel_nodes_poi'].str.split('~')
df_afstandn2['x'] = df_afstandn2['split'].apply(lambda x: x[0]).astype('float').round().astype('int')
df_afstandn2['y'] = df_afstandn2['split'].apply(lambda x: x[1]).astype('float').round().astype('int')
df_afstandn2['type'] = df_afstandn2['split'].apply(lambda x: x[2])
verblijfsobjecten = df_afstandn2[df_afstandn2['type'] != 'afval_cluster']
verblijfsobjecten['bag'] = verblijfsobjecten['split'].apply(lambda x: x[3]).astype('int64')


In [ ]:
temp = dfob.set_index(['bag', 'x', 'y']).join(verblijfsobjecten.set_index(['bag', 'x', 'y']), how='outer').reset_index()

In [ ]:
df_afstandn = get_distance_matrix()

In [ ]:
joined= temp.set_index('s1_afv_nodes').join(df_afstandn.set_index('naar_s1_afv_nodes'), how='outer')
joined = joined.reset_index()[['van_s1_afv_nodes', 'index', 'afstand', 'count']].\
         rename(columns={'index':'naar_s1_afv_nodes'}).sort_values(by='afstand').\
         reset_index().drop(['index'],axis=1).dropna()

In [ ]:
joined

In [ ]:
def distance_matrix_with_counts():
    """
    Function that tries to match table with addresses per poi with 
    information and subsequently with distance matrix to give back
    a distance matrix with the amount of households per addres poi.
    """
    
    dfob= get_dataframe("""
                    SELECT bk_votpand_cluster, COUNT(*)
                    FROM proj_afval_netwerk.rel_votpand_cluster_verblijfsobject
                    GROUP BY bk_votpand_cluster
                    """)
    
    dfob['split'] = dfob['bk_votpand_cluster'].str.split('~')
    dfob['bag'] = dfob['split'].apply(lambda x: x[0]).astype('int64')
    dfob['x'] = dfob['split'].apply(lambda x: x[1]).astype('float').round().astype('int')
    dfob['y'] = dfob['split'].apply(lambda x: x[2]).astype('float').round().astype('int')
    dfob = dfob.drop(['split'], axis=1)
    
    df_afstandn2 = get_dataframe("""
                                SELECT *
                                FROM proj_afval_netwerk.afv_rel_nodes_poi
                                """)
    
    df_afstandn2['split'] = df_afstandn2['bk_afv_rel_nodes_poi'].str.split('~')
    df_afstandn2['x'] = df_afstandn2['split'].apply(lambda x: x[0]).astype('float').round().astype('int')
    df_afstandn2['y'] = df_afstandn2['split'].apply(lambda x: x[1]).astype('float').round().astype('int')
    df_afstandn2['type'] = df_afstandn2['split'].apply(lambda x: x[2])
    verblijfsobjecten = df_afstandn2[df_afstandn2['type'] != 'afval_cluster']
    verblijfsobjecten['bag'] = verblijfsobjecten['split'].apply(lambda x: x[3]).astype('int64')
    
    temp = dfob.set_index(['bag', 'x', 'y']).join(verblijfsobjecten.set_index(['bag', 'x', 'y']), how='outer').reset_index()
    df_afstandn = get_distance_matrix()
    joined= temp.set_index('s1_afv_nodes').join(df_afstandn.set_index('naar_s1_afv_nodes'), how='outer')
    joined = joined.reset_index()[['van_s1_afv_nodes', 'index', 'afstand', 'count']].\
             rename(columns={'index':'naar_s1_afv_nodes'}).sort_values(by='afstand').\
             reset_index().drop(['index'],axis=1).dropna()
    
    return joined

In [ ]:
joined = distance_matrix_with_counts()

### Try out

In [2]:
all_households, rel_poi_df, joined, df_afstandn2 = initial_loading(use_count=True)

API data loaded
a
DB relation POIs loaded
b
Table all households created
API and DB joined
containers per cluster determined


../Code\loading_data.py:219: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  verblijfsobjecten['bag'] = verblijfsobjecten['split'].apply(lambda x: x[3]).astype('int64')


distance matrix loaded


In [5]:
df_afstandn2

,van_s1_afv_nodes,naar_s1_afv_nodes,afstand,count
0,855922.0,512375.0,5.46,1.0
1,734016.0,927554.0,6.42,4.0
2,827785.0,556399.0,7.15,4.0
3,637963.0,730978.0,7.19,4.0
4,581294.0,658854.0,7.26,3.0
...,...,...,...,...
9633824,758819.0,512435.0,999.99,5.0
9633825,659257.0,816941.0,999.99,3.0
9633826,526282.0,702805.0,999.99,20.0
9633827,569984.0,632572.0,999.99,3.0


In [6]:
joined_cluster_distance = joined.set_index('s1_afv_nodes').join(df_afstandn2.set_index('van_s1_afv_nodes')).reset_index().rename(columns={'index': 'van_s1_afv_nodes'})

In [20]:
good_result_rich = add_shortest_distances_to_all_households(all_households, joined_cluster_distance, count=True)
good_result_rich = good_result_rich[good_result_rich['uses_container']]
good_result_rich

,cluster_x,cluster_y,uses_container,count,poi_rest,rest_afstand,poi_plastic,plastic_afstand,poi_papier,papier_afstand,poi_glas,glas_afstand,poi_textiel,textiel_afstand
naar_s1_afv_nodes,,,,,,,,,,,,,,
483489,119757,488187,True,4.0,527392.0,49.41,718501.0,297.63,527392.0,49.41,527392.0,49.41,831157.0,229.49
483491,128330,485171,True,1.0,603871.0,45.37,843453.0,87.15,563386.0,191.53,563386.0,191.53,563386.0,191.53
483492,116782,485130,True,3.0,790214.0,130.99,854113.0,203.94,579259.0,195.81,854113.0,203.94,743514.0,305.62
483497,126721,481159,True,91.0,785277.0,51.68,520452.0,588.29,689543.0,489.10,689543.0,489.10,958308.0,692.23
483498,119059,486577,True,4.0,637963.0,12.99,800188.0,122.73,714362.0,292.18,714362.0,292.18,860040.0,477.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
966947,120116,487957,True,4.0,887501.0,108.30,887501.0,108.30,772674.0,162.30,772674.0,162.30,772674.0,162.30
966950,118656,486046,True,3.0,765331.0,23.82,682426.0,218.60,892569.0,137.51,892569.0,137.51,486758.0,152.65
966954,119052,481924,True,1.0,915296.0,22.52,919134.0,250.59,619463.0,104.80,619463.0,104.80,919134.0,250.59


In [45]:
aansluitingen = create_aansluitingen(good_result_rich, joined_cluster_distance, use_count=True)

In [60]:

def calculate_penalties(good_result, aansluitingen, use_count=False):
    """
    This function calculates all the penalties associated with the candidate
    solution. It does this by calculating the number of times all constraints
    are violated and applies the weighing that is associated with all these
    violations

    Input:
    dataframe good_result containing per adress or adress poi the distance
    to the nearest container for all fractions.
    dataframe aansluitingen containing for all clusters the amount of containers
    per fraction, the amount of people using these containers and the percentage
    of occupancy compared to the norm

    Output:
    The sum of all different penalties as a single float
    """
    
    penalty1 = good_result[good_result['rest_afstand'] > 100]
    penalty2 = good_result[good_result['plastic_afstand'] > 150]
    penalty3 = good_result[good_result['papier_afstand'] > 150]
    penalty4 = good_result[good_result['glas_afstand'] > 150]
    penalty5 = good_result[good_result['textiel_afstand'] > 300]
    penalty6 = aansluitingen[aansluitingen['rest_perc'] > 100]
    penalty7 = aansluitingen[aansluitingen['plastic_perc'] > 100]
    penalty8 = aansluitingen[aansluitingen['papier_perc'] > 100]
    penalty9 = aansluitingen[aansluitingen['glas_perc'] > 100]
    penalty10 = aansluitingen[aansluitingen['textiel_perc'] > 100]
    
    if not use_count:
        penalty1_sum = (penalty1['rest_afstand'].sum() - 100 * penalty1.shape[0])/good_result.shape[0] * 0.35
        penalty2_sum = (penalty2['plastic_afstand'].sum() - 150 * penalty2.shape[0])/good_result.shape[0] * 0.25
        penalty3_sum = (penalty3['papier_afstand'].sum() - 150 * penalty3.shape[0])/good_result.shape[0] * 0.2
        penalty4_sum = (penalty4['glas_afstand'].sum() - 150 * penalty4.shape[0])/good_result.shape[0] * 0.15
        penalty5_sum = (penalty5['textiel_afstand'].sum() - 300 * penalty5.shape[0])/good_result.shape[0] * 0.05
        penalty6_sum = (penalty6['poi_rest'] - (penalty6['rest'] * 100)).sum()/ good_result.shape[0] * 0.35 * 1000
        penalty7_sum = (penalty7['poi_plastic'] - (penalty7['plastic'] * 200)).sum()/ good_result.shape[0] * 0.25 * 1000
        penalty8_sum = (penalty8['poi_papier'] - (penalty8['papier'] * 200)).sum()/ good_result.shape[0] * 0.2 * 1000
        penalty9_sum = (penalty9['poi_glas'] - (penalty9['glas'] * 200)).sum()/ good_result.shape[0] * 0.15 * 1000
        penalty10_sum = (penalty10['poi_textiel'] - (penalty10['textiel'] * 750)).sum()/ good_result.shape[0] * 0.05 * 1000
        
    else:
        penalty1_sum = (penalty1['rest_afstand'].sum() - 100 * penalty1['count'].sum())/good_result['count'].sum() * 0.35
        penalty2_sum = (penalty2['plastic_afstand'].sum() - 150 * penalty2['count'].sum())/good_result['count'].sum() * 0.25
        penalty3_sum = (penalty3['papier_afstand'].sum() - 150 * penalty3['count'].sum())/good_result['count'].sum() * 0.2
        penalty4_sum = (penalty4['glas_afstand'].sum() - 150 * penalty4['count'].sum())/good_result['count'].sum() * 0.15
        penalty5_sum = (penalty5['textiel_afstand'].sum() - 300 * penalty5['count'].sum())/good_result['count'].sum() * 0.05
        penalty6_sum = (penalty6['poi_rest'] - (penalty6['rest'] * 100)).sum()/ good_result['count'].sum() * 0.35 * 1000
        penalty7_sum = (penalty7['poi_plastic'] - (penalty7['plastic'] * 200)).sum()/ good_result['count'].sum() * 0.25 * 1000
        penalty8_sum = (penalty8['poi_papier'] - (penalty8['papier'] * 200)).sum()/ good_result['count'].sum() * 0.2 * 1000
        penalty9_sum = (penalty9['poi_glas'] - (penalty9['glas'] * 200)).sum()/ good_result['count'].sum() * 0.15 * 1000
        penalty10_sum = (penalty10['poi_textiel'] - (penalty10['textiel'] * 750)).sum()/ good_result['count'].sum() * 0.05 * 1000
        
    total_penalties = sum([penalty1_sum, penalty2_sum, penalty3_sum, penalty4_sum, penalty5_sum,\
                           penalty6_sum, penalty7_sum, penalty8_sum, penalty9_sum, penalty10_sum])
    return total_penalties


In [62]:
calculate_penalties(good_result_rich, aansluitingen, use_count=True)

295.3414480371864